In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [3]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.8 MB/s eta 0:00:00


In [4]:
pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.5 MB/s eta 0:00:00


In [5]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=31a05a99dae36fe2700590b54a5072befebfb91bed08afd35260c0e660e94a96
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [6]:
pip install Pillow

In [7]:
pip install pytesseract

In [8]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,766 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [9]:
!pip install pytesseract


In [12]:
!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=7dabf7059432d05c4b6b25c9c50ca1b0798913fe83ca44666c78dc5e53f09e68
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


In [16]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import glob
import pandas as pd
from PyPDF2 import PdfReader
from docx import Document
from pptx import Presentation
from PIL import Image
import pytesseract
import json
from odf.opendocument import load
from odf.text import P

# Use the correct local path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/odt test corpus'

# Ensure the directory exists
os.makedirs(folder_path, exist_ok=True)

# List all files in the folder
files = glob.glob(os.path.join(folder_path, '**'), recursive=True)

# Initialize a variable to store concatenated content
corpus = ""

# Function to add a delimiter and metadata
def add_delimiter(filename):
    return f"\n\n---\nFILE: {filename}\n---\n\n"

# Define processing functions for each file type
def process_pdf(file, corpus):
    reader = PdfReader(file)
    for page in reader.pages:
        corpus += page.extract_text() + "\n"
    return corpus

def process_csv(file, corpus):
    df = pd.read_csv(file)
    corpus += df.to_string() + "\n"
    return corpus

def process_parquet(file, corpus):
    df = pd.read_parquet(file)
    corpus += df.to_string() + "\n"
    return corpus

def process_docx(file, corpus):
    doc = Document(file)
    for para in doc.paragraphs:
        corpus += para.text + "\n"
    return corpus

def process_odt(file, corpus):
    doc = load(file)
    for paragraph in doc.getElementsByType(P):
        corpus += str(paragraph) + "\n"
    return corpus

def process_pptx(file, corpus):
    prs = Presentation(file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                corpus += shape.text + "\n"
    return corpus

def process_image(file, corpus):
    text = pytesseract.image_to_string(Image.open(file))
    corpus += text + "\n"
    return corpus

def process_code(file, corpus):
    with open(file, 'r') as f:
        corpus += f.read() + "\n"
    return corpus

def process_ipynb(file, corpus):
    with open(file, 'r') as f:
        notebook = json.load(f)
        for cell in notebook['cells']:
            if cell['cell_type'] == 'markdown':
                corpus += ''.join(cell['source']) + "\n"
            elif cell['cell_type'] == 'code':
                corpus += ''.join(cell['source']) + "\n"
    return corpus

# Function to process files based on their extension
def process_files(files, extension, processing_function, corpus):
    matching_files = [f for f in files if f.endswith(extension)]
    if matching_files:
        for file in matching_files:
            print(f"Processing file: {file}")  # Debug print
            corpus += add_delimiter(file)
            corpus = processing_function(file, corpus)
    return corpus

# Process each type of file if present in the folder
corpus = process_files(files, '.pdf', process_pdf, corpus)
corpus = process_files(files, '.csv', process_csv, corpus)
corpus = process_files(files, '.parquet', process_parquet, corpus)
corpus = process_files(files, '.docx', process_docx, corpus)
corpus = process_files(files, '.odt', process_odt, corpus)
corpus = process_files(files, '.pptx', process_pptx, corpus)
corpus = process_files(files, ('.png', '.jpg', '.jpeg'), process_image, corpus)  # Handle multiple image extensions
corpus = process_files(files, ('.py', '.js', '.html', '.css'), process_code, corpus)  # Handle multiple code file extensions
corpus = process_files(files, '.ipynb', process_ipynb, corpus)

# Placeholder for video file handling
video_files = [f for f in files if f.endswith(('.mp4', '.avi', '.mkv'))]
for file in video_files:
    print(f"Processing video file: {file}")  # Debug print
    corpus += add_delimiter(file)
    corpus += f"Video file: {file}\n"

# Check the length of the corpus to ensure it has content
print(f"Length of corpus: {len(corpus)}")  # Debug print

# Save the consolidated corpus into a text file
corpus_path = os.path.join(folder_path, 'full_corpus.txt')
with open(corpus_path, 'w') as f:
    f.write(corpus)

print(f'Consolidated corpus saved at {corpus_path}')

Mounted at /content/drive
Processing file: /content/drive/MyDrive/odt test corpus/Muscle_Gain_and_Fat_Loss_Plan.odt
Processing file: /content/drive/MyDrive/odt test corpus/ROCK data architecture (2).odt
Length of corpus: 36739
Consolidated corpus saved at /content/drive/MyDrive/odt test corpus/full_corpus.txt


This code mounts your Google Drive in Google Colab and reads files from a specified folder. It processes different types of files, such as PDFs, CSVs, DOCX, PPTX, images (using OCR), code files, and Jupyter Notebooks. For each file, it extracts the text content and adds it to a consolidated text file. The script handles each file type separately and only processes the files that are present in the folder. Finally, it saves all the collected text into a single text file in the same folder on Google Drive.